# ABS Retail Trade 8501

Note: the ABS usually releases the Retail Trade data in tranches, with the second tranch delivered about a week after the first.

## Python set-up

In [1]:
# system imports
from typing import cast

# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    fix_abs_title,
    AbsLandingPage,
    get_abs_data,
    get_fs_constants,
    metacol,
    get_plot_constants,
)
from plotting import (
    calc_growth,
    clear_chart_dir,
    plot_covid_recovery,
    plot_growth_finalise,
    recalibrate_series,
    recalibrate,
    set_chart_dir,
    finalise_plot,
)
from decompose import decompose
from utility import qtly_to_monthly


# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999

# display charts in the notebook ...
SHOW = False

## Get data from the ABS

### Retail trade data

In [2]:
landing_page = AbsLandingPage(
    theme="industry",
    parent_topic="retail-and-wholesale-trade",
    topic="retail-trade-australia",
)
abs_dict = get_abs_data(landing_page)
source, CHART_DIR, Cat_ID, meta = get_fs_constants(abs_dict, landing_page)
RECENT, plot_times, plot_tags = get_plot_constants(meta)

In [3]:
# Check latest monthly date
print(abs_dict["1"].index[-1])

2024-03


In [4]:
# check latest quarterly date
if "7" in abs_dict:
    print(abs_dict["7"].index[-1])

2024Q1


In [5]:
# Monthly tables are numbered 1 through 4
# Quarterly tables are numbered 5 and following.
print(meta["Table"].unique())

['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12' '13' '14' '15' '16'
 '17' '18' '19' '20' '21' '22' '23']


### CPI deflator

In [6]:
def get_cpi_deflator() -> tuple[pd.Series, str, str]:
    """Get CPI deflator, including a forward projection."""

    # get CPI tables from the ABS
    cpi_landing_page = AbsLandingPage(
        theme="economy",
        parent_topic="price-indexes-and-inflation",
        topic="consumer-price-index-australia",
    )
    cpi_dict = get_abs_data(cpi_landing_page)
    _, _, cpi_catalogue_id, cpi_meta = get_fs_constants(cpi_dict, cpi_landing_page)

    # extract quarterly All Groups CPI index from the ABS data
    cpi_index = "Index Numbers ;  All groups CPI ;  Australia ;"
    cpi_table = "2"
    cpi_row = cpi_meta[
        (cpi_meta[metacol.did] == cpi_index) & (cpi_meta[metacol.table] == cpi_table)
    ].iloc[0]
    cpi_id = cpi_row[metacol.id]
    cpi_q = cpi_dict[cpi_table][cpi_id]

    # rebase quarterly CPI index to one
    cpi_q = cpi_q / cpi_q.iloc[-1]

    # calculate recent monthly CPI growth
    months_per_quarter = 3
    quarters = 2
    period = months_per_quarter * quarters  # in months
    monthly_growth = (cpi_q.iloc[-1] / cpi_q.iloc[-1 - quarters]) ** (1 / period)

    # create a monthly CPI index ...
    cpi_m = qtly_to_monthly(cpi_q)

    # extrapolate monthly index forward ...
    additional_months = 4
    for _ in range(additional_months):
        cpi_m[cpi_m.index[-1] + 1] = cpi_m.iloc[-1] * monthly_growth

    print()
    print(f"Monthly growth: {monthly_growth}")
    print(f"Projected CPI: {cpi_m.tail(12)}")
    return cpi_m, cpi_catalogue_id, cpi_table

In [7]:
cpi_deflator, CPI_CAT_ID, CPI_TABLE = get_cpi_deflator()


Monthly growth: 1.0025702716008655
Projected CPI: Series ID
2023-08    0.980835
2023-09    0.984716
2023-10    0.986657
2023-11    0.988598
2023-12    0.990539
2024-01    0.993692
2024-02    0.996846
2024-03    1.000000
2024-04    1.002570
2024-05    1.005147
2024-06    1.007731
2024-07    1.010321
Freq: M, Name: A2325846C, dtype: float64


## Plot - retail turnover by industry

### Plotting initialisation / constants

In [8]:
def chart_settings(chart_dir: str = CHART_DIR) -> None:
    """Set up charting parameters."""

    set_chart_dir(chart_dir)
    clear_chart_dir(chart_dir)
    plt.style.use("fivethirtyeight")


chart_settings()
LONG_LABEL = 35  # characters

### Main plotting loop

In [14]:
def produce_charts() -> None:
    """Produce charts."""

    series_type = "Seasonally Adjusted"
    rows = meta[
        meta[metacol.did].str.contains("Turnover", regex=False)
        & meta[metacol.unit].str.contains("$", regex=False)
        & (meta[metacol.stype] == series_type)
    ]

    recent = plot_times[1]
    for _, row in rows.iterrows():
        # get the data for wacg plotable item
        series_id, units, table = row[metacol.id], row[metacol.unit], row[metacol.table]
        data = abs_dict[table]
        series, units = recalibrate_series(data[series_id], units)
        series_freq = cast(pd.PeriodIndex, series.index).freqstr
        check = pd.Period("2017-01-01", freq=series_freq)
        check_series = series[series.index > check]
        if len(check_series) != len(check_series.dropna()):
            print(f"Odd or missing data: {row[metacol.did]}")
            continue

        # plot the data
        headline = "!"
        pre_tag = headline if table in ["1", "7"] else ""
        series.name = "Series"
        monthly = "Monthly"
        frequency = {"Q": "Quarterly", "M": monthly}[series_freq[0]]
        lfooter = f"Australia. {series_type.capitalize()}. "
        current_prices = "Current Prices. "
        lfooter += "Current Prices. " if frequency == monthly else ""
        title, lfooter = fix_abs_title(row[metacol.did], lfooter)
        separator = "\n" if len(title) > LONG_LABEL else " "
        title = (
            f"{frequency} Retail Trade:{separator}"
            # Move "Turnover" to the y-axis label.
            f"{title.replace('Turnover', '').strip()}"
        )
        plot_covid_recovery(
            series,
            title=title,
            ylabel=f"Turnover {units}",
            pre_tag=pre_tag,
            rfooter=f"{source} T{table}",
            lfooter=lfooter,
            show=pre_tag == headline and SHOW,
        )

        if pre_tag == headline and frequency == monthly:
            cpi_adj_series = (series / cpi_deflator).dropna()
            cpi_adj_series.name = "series"
            plot_covid_recovery(
                cpi_adj_series,
                title=title,
                ylabel=f"Turnover {units} (CPI Adj)",
                pre_tag=pre_tag,
                tags="cpi-adjusted",
                rfooter=f"{source} T{table} {CPI_CAT_ID} T{CPI_TABLE}",
                lfooter=lfooter.replace(current_prices, "CPI adjusted prices. "),
                lheader="CPI Adjusted using All Groups CPI",
                show=pre_tag == headline and SHOW,
            )

        if pre_tag == headline:
            # Only do growth charts on headline items
            growth = calc_growth(series)
            plot_growth_finalise(
                *growth,
                from_=pd.Period(
                    recent, freq=cast(pd.PeriodIndex, series.index).freqstr[0]
                ),
                title=f"Growth in {title.title()}",
                pre_tag=headline,
                rfooter=f"{source} T{table}",
                lfooter=lfooter,
                annotate=True,
                show=SHOW,
            )


produce_charts()

Odd or missing data: Turnover ;  Queensland ;  Liquor retailing ;
Odd or missing data: Turnover ;  Queensland ;  Other specialised food retailing ;
Odd or missing data: Turnover ;  South Australia ;  Liquor retailing ;
Odd or missing data: Turnover ;  South Australia ;  Other specialised food retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Liquor retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Other specialised food retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Hardware, building and garden supplies retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Household goods retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Footwear and other personal accessory retailing ;
Odd or missing data: Turnover ;  Tasmania ;  Clothing, footwear and personal accessory retailing ;
recalibrate(): All NaN data.
Odd or missing data: Turnover ;  Tasmania ;  Department stores ;
Odd or missing data: Turnover ;  Tasmania ;  Other retailing n.e.c. ;
Odd or missing data: Tu

### Seasonal adjustment check

In [10]:
def seasonal() -> None:
    """Produce alternate seasonal adjustment charts."""

    table = "1"
    meta1 = meta[meta.Table == table]
    df = abs_dict[table]
    df = df[df.index.year >= (df.index[-1].year - 5)]

    for did in meta1[metacol.did].unique():

        # extract key data
        data = []
        for seasonal in "Original", "Seasonally Adjusted":
            row = meta1[
                (meta1[metacol.did] == did) & (meta1[metacol.stype] == seasonal)
            ]
            id_ = row[metacol.id].values[0]
            units = row[metacol.unit].values[0]
            data.append(df[id_])
        factor = pd.DataFrame(data[0] / data[1], columns=["Factor"])
        factor["Year"] = factor.index.year
        factor["Month"] = factor.index.month
        data_table = factor.pivot(values="Factor", index="Month", columns="Year")
        data_table.columns = data_table.columns.astype(str) + " ABS"

        # Let's check how the decomp looks
        # based on a more recent assessment of seasonality
        # NOTE: highly speculative analysis.
        post_covid = data[0][data[0].index >= pd.Period("2021-11", freq="M")]
        decomp = decompose(
            post_covid,
            constant_seasonal=True,
            arima_extend=False,
        )
        months = decomp["Seasonal Weights"].groupby(decomp.index.month).mean()
        data_table["Recent Alternate"] = months

        # chart tidy-up
        title, lfooter = fix_abs_title(did, "")
        title = title.replace("Turnover", "").strip()
        title = "\n" + title if title else title + " - Total"
        adj = 0.02
        ylim = data_table.min().min() - adj, data_table.max().max() + adj

        # plot
        ax = data_table.plot.bar()
        finalise_plot(
            ax,
            title=f"Seasonal factors - Monthly Retail Turnover{title}",
            ylabel="Divisor",
            xlabel="Month",
            rfooter=f"{source} T{table}",
            lfooter=f"Australia: {lfooter}",
            ylim=ylim,
            legend={"fontsize": "x-small", "ncol": 2},
            show=SHOW,
        )

        # And an ABS  comparative plot.
        alternate = pd.DataFrame(
            [data[1], decomp["Seasonally Adjusted"]],
            index=["ABS seasonally adjusted", "Alternate seasonal adjustment"],
        ).T
        alternate, units = recalibrate(alternate, units)
        ax = alternate.plot(label="ABS Seasonally Adjusted", lw=2.5)
        finalise_plot(
            ax,
            title=f"Alt. Seas. Adj. - Monthly Retail Turnover{title}",
            ylabel=f"Turnover {units}",
            xlabel=None,
            rfooter=f"{source} T{table}",
            lfooter=f"Australia: {lfooter}",
            legend={"fontsize": "x-small", "ncol": 2},
            show=SHOW,
        )

In [11]:
seasonal()

## Finished

In [12]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Tue May 07 2024 12:44:51

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.22.2

matplotlib: 3.8.4
pandas    : 2.2.2

Watermark: 2.4.3



In [13]:
print("Finished")

Finished
